In [645]:
import numpy as np
import pandas as pd
import re
import jieba

In [646]:
data0=pd.read_excel("./data/dataset/dataset0.xls")
data=data0.sample(frac=1.0)
y=data.tcms_code
outm=data.iloc[:,8:]
context=data.contxt
diag=data.diag
data.head()

,hid,id,pid,contxt,diag,diag_code,tcm_syn,tcms_code,tan,feng,han,ren,qixu,yinxu,fei,pi,shen
10519,2,10520,201930154,咳嗽，咳痰，气促3个月。 望之少神、表情痛苦，面色无华，形体适中；不能久立、精神不振、发育正...,支气管扩张症伴感染,6,痰热壅肺,8,1,0,0,1,0,0,1,0,0
5378,2,5379,201839501,反复胸闷、气促1年，再发加重1天。 望之少神、表情痛苦，面色少华，形体适中；行动自如、精神不...,支气管哮喘,7,寒哮,5,0,0,1,0,0,0,1,0,0
10117,1,10118,1500265,鼻塞流涕咳嗽伴胸中不适十六天 面色自然，双目有神。语声清晰，呼吸平稳，未闻及特殊气味。舌质红...,急性支气管炎,2,痰热壅肺,8,1,0,0,1,0,0,1,0,0
11625,1,11626,1609618,反复咳嗽咳痰气喘八年，加重一天。 痴呆状，气喘明显，喉中痰鸣，排痰不能，阵发性咳嗽。舌质偏淡...,慢性阻塞性肺病伴感染,5,痰浊阻肺,9,1,0,0,0,0,0,1,0,0
8211,1,8212,1501574,咳嗽咳痰伴发热十余天。 :该患者以咳嗽咳痰发热为主诉，结合患者舌脉，舌质红，苔黄，脉滑，,肺部感染,1,痰热壅肺,8,1,0,0,1,0,0,1,0,0


In [647]:
print(data.shape)
print(y.shape)
print(outm.shape)
print(context.shape)
print(diag.shape)
print(set(data.diag_code))

(14075, 17)
(14075,)
(14075, 9)
(14075,)
(14075,)
{0, 1, 2, 3, 4, 5, 6, 7}


In [648]:
data0_ex=pd.read_excel("./data/dataset/external0.xls")
data_ex=data0_ex.sample(frac=1.0)
y_ex=data_ex.tcms_code
outm_ex=data_ex.iloc[:,8:]
context_ex=data_ex.contxt
diag_ex=data_ex.diag
data_ex.head()

,hid,id,pid,contxt,diag,diag_code,tcm_syn,tcms_code,tan,feng,han,ren,qixu,yinxu,fei,pi,shen
343,2,3704,201921677,反复咳嗽咳痰4月余，加重10余天。 望之少神、表情痛苦，面色少华，形体适中；动则气促、精神不...,肺部感染,1,风寒袭肺,3,0,1,1,0,0,0,1,0,0
492,2,4500,201927871,咳嗽、畏寒7天，发热3天 望之有神、表情正常，面色荣润，形体适中；行动自如、精神良好、发育正...,急性支气管炎,2,风热犯肺,4,0,1,0,1,0,0,1,0,0
361,2,3593,201900114,咳嗽、咳痰、气促8天，加重1天。 望之少神、表情痛苦，面色少华，形体适中；动则气促、精神不振...,肺部感染,1,风寒袭肺,3,0,1,1,0,0,0,1,0,0
78,1,939,1506437,确诊肺癌一年余，胸闷伴背痛一周。 面色自然，形体正常，双目有神，语声清晰，呼吸平稳，未闻及特...,肺癌,0,肺脾气虚,0,0,0,0,0,1,0,1,1,0
536,2,5340,201849028,反复喘息气促、咳嗽咳痰40余年，再发加重7天。 望之少神、表情正常，面色荣润，形体适中；不能...,支气管哮喘,7,寒哮,5,0,0,1,0,0,0,1,0,0


In [649]:
print(data_ex.shape)
print(y_ex.shape)
print(outm_ex.shape)
print(context_ex.shape)
print(diag_ex.shape)
print(set(data_ex.diag_code))

(1000, 17)
(1000,)
(1000, 9)
(1000,)
(1000,)
{0, 1, 2, 3, 4, 5, 6, 7}


In [650]:
dictfile="./data/tcm_all_dict_2.txt"
stopwords="./data/stopwords.txt"
jieba.load_userdict(dictfile)   

In [651]:
def file2doc1(infile,dictfile):    
    jieba.load_userdict(dictfile)                     
    doc = [w for x in codecs.open(file_name, 'r', 'utf-8').readlines() for w in jieba.cut(x.strip())]
    return doc

In [652]:
def load_stopword(filename):  
    stopwords = [line.strip() for line in open(filename, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [653]:
def extract_chn(data):
    pattern="[\u4e00-\u9fa5]"
    regex=re.compile(pattern)
    res=regex.findall(data)
    #return "".join(res)
    return res

In [654]:
def drop_alnum(astr):
    drop_str="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    str1=astr
    for ch in str1:
        if ch in drop_str:
            str1=str1.replace(ch,"")
    return str1

In [655]:
def get_corpus(context):
    corpus=[]    
    
    for sent in context:
        sent=sent+"；"
        sent=del_neg_txt1(sent) #drop negative words  
        sent=del_neg_txt2(sent)
        sent.strip("；")
        strlist=list(jieba.cut(sent))
        strlist=sorted(set(strlist),key=strlist.index)
        corpus.append(strlist)
        
    return corpus

In [656]:
def get_corpus1(doc,stopwords):    
    stopword_list=load_stopword(stopwords)   
    #r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'   #drop letters, numbers and sepcial chars
    out=[]
    
    for sent in doc:        
        doc_stopword=[y for y in sent if y not in stopword_list] #drop stopword
        res=[]
        for y in doc_stopword:
            #res.append(re.sub(r1,"",y))   #drop letters, numbers and sepcial chars
            y=drop_alnum(y)
            res.append(y)  #drop letters, numbers and sepcial chars
        out.append(res)
    return out

In [657]:
def get_corpus2(corpus,diagnosis):        
    out=list()  
    
    LEN=len(corpus)
    diaglist=list(diagnosis)
    for i in range(LEN):
        inlist=list()
        inlist.extend(corpus[i])        
        inlist.append(diaglist[i])
        out.append(inlist)        
    return out

In [658]:
def del_neg_txt1(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="无"   
    item1=["；","，","舌"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [659]:
def del_neg_txt2(contxt):
    txt=contxt
    endtxt=""
    txttmp=""
    item0="未"
    item1=["；","，","有"] 
    #item1=["；","，"]

    for i in range(len(contxt)):
        if txt.find(item0)>0:
            start=txt.find(item0)
            subtxt1=txt[:start]
            txttmp+=subtxt1
            #print(txttmp+"\n")
            subtxt=txt[start:]
            #print(subtxt+"\n")
    
            for sign in item1:        
                if subtxt.find(sign)>0:        
                    end=subtxt.find(sign)        
                    break 
            txt=subtxt[end:]
            #print(txt+"\n")
        endtxt=txt
        
    txttmp+=endtxt
    #print(txttmp+"\n")
    return txttmp

In [661]:
atxt=get_corpus(context)
atxt_ex=get_corpus(context_ex)
#print(atxt)
cps=get_corpus1(atxt, stopwords)
cps_ex=get_corpus1(atxt_ex,stopwords)
#print(len(cps))
cps1=get_corpus2(cps,diag)
#i=0
#for sent in cps:
    #print(i, sent)
   #i+=1

In [520]:
print(len(cps),len(cps_ex))

14075 1000


In [521]:
#for i, sent in enumerate(cps):
    #print(i, sent)

In [522]:
import gensim
import codecs
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [523]:
#model_w2v= gensim.models.KeyedVectors.load_word2vec_format("./model/w2v_122g.bin",binary=True) #window=5, min_count=5,size=128, workers=4 122g-corpus
model_w2v= gensim.models.Word2Vec.load("./model_w2v/w2v_wx") #window=5, min_count=5,size=256, workers=4

d:\code\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
d:\code\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
d:\code\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [524]:
def get_vectors(corpus,model):
    vectors=[]
    
    for sent in corpus:    
        sentvec=[]
        for word in sent:   
            if word in model.wv.index2word:
                sentvec.append(model[word])
        vectors.append(sentvec)            
    return vectors

In [525]:
import time
st1=time.time()
vectors_w2v=get_vectors(cps,model_w2v)
vectors_w2v_ex=get_vectors(cps_ex,model_w2v)
st2=time.time()
elapse=(st2-st1)/60.0
print("elapse time : %f min"%elapse)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


elapse time : 44.346143 min


In [180]:
#word2vec cnn 1

In [285]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [286]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [287]:
y=data.tcms_code
outm=data.iloc[:,8:]
x_ex=cps_ex
y_ex=data_ex.tcms_code

all_texts=cps
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))
datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y),num_classes=None)

ex_texts=cps_ex
tokenizer1=Tokenizer()
tokenizer1.fit_on_texts(ex_texts)
sequences1=tokenizer.texts_to_sequences(ex_texts)
datatxt_ex=pad_sequences(sequences1,maxlen=MAX_SEQUENCE_LENGTH)
labels_ex= to_categorical(np.asarray(y_ex),num_classes=None)
#print('Shape of data tensor:', datatxt.shape)
#print('Shape of label tensor:', labels.shape)
#print(word_index)
y_ex=labels_ex

In [288]:
from sklearn.model_selection import train_test_split
x=datatxt
y1=labels
x_train, x_test, y_train, y_test=train_test_split(x,y1,test_size=0.2,random_state=None)

In [289]:
# text - cnn model

In [290]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
#EMBEDDING_DIM = 128

In [291]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 64, 256)           600576    
_________________________________________________________________
dropout_13 (Dropout)         (None, 64, 256)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 32, 512)           0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 256)               4194560   
_________________________________________________________________
dense_26 (Dense)             (None, 10)                2570      
Total para

In [292]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_split=0.2, shuffle=True) 
#score1 = model.evaluate(x_train,y_train, verbose=0)
score2 = model.evaluate(x_test,y_test, verbose=0)
score_ex = model.evaluate(datatxt_ex,y_ex, verbose=0)
#score_ex=model.evaluate(x_ex_test,y_ex_test, verbose=0)
y_predict=model.predict(x_test)
y_pred_ex=model.predict(datatxt_ex)
y_predict1=np.argmax(y_predict,axis=1)
y_predict2=np.argmax(y_pred_ex,axis=1)
print(score2[1],score_ex[1])

Train on 9008 samples, validate on 2252 samples
Epoch 1/4
9008/9008 [==============================] - 27s 3ms/step - loss: 0.9660 - acc: 0.6842 - val_loss: 0.4273 - val_acc: 0.8655
Epoch 2/4
9008/9008 [==============================] - 26s 3ms/step - loss: 0.2253 - acc: 0.9288 - val_loss: 0.2451 - val_acc: 0.9241
Epoch 3/4
9008/9008 [==============================] - 26s 3ms/step - loss: 0.1157 - acc: 0.9648 - val_loss: 0.2452 - val_acc: 0.9307
Epoch 4/4
9008/9008 [==============================] - 27s 3ms/step - loss: 0.0679 - acc: 0.9782 - val_loss: 0.2457 - val_acc: 0.9352
0.9410301950642737 0.955


In [293]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [294]:
# evaluation of performance on external dataset

In [295]:
y_labels1_ex = to_categorical(np.asarray(y_predict2),num_classes=None) #real to vector
#ym=outm
#y_m=np.argmax(y_labels1,axis=1)
#print("y_m",y_m)

y_pred2_ex=y_predict2  #vector to scalar
y_p1_ex=np.argmax(y_ex,axis=1) #vecor to scalar
#print(y_p1)

print("acc1:",metrics.accuracy_score(y_p1_ex, y_pred2_ex))
print("prec1:",metrics.precision_score(y_p1_ex, y_pred2_ex, average='weighted'))
print("recall1:",metrics.recall_score(y_ex, y_labels1_ex, average='weighted'))
print("fscore1:",metrics.f1_score(y_ex, y_labels1_ex, average='weighted'))

acc1: 0.955
prec1: 0.9575435297650285
recall1: 0.955
fscore1: 0.9549943922029115


In [296]:
# evaluation 95%CI for metrics

In [297]:
y_true_ex=pd.DataFrame(y_p1_ex,columns=["true"])
y_pred1_ex=pd.DataFrame(y_pred2_ex,columns=["pred"])
y_true1_ex=y_true_ex.reset_index(drop=True)
#print(y_pred1)
#print(y_true1)

y_true_pred_ex=pd.concat([y_true1_ex,y_pred1_ex],axis=1)
y_true_pred_ex.columns=["true","pred"]
#print(y_true_pred)

acc_list=list()
prec_list=list()
rec_list=list()
f1_list=list()
for i in range(1000):
    resample=y_true_pred_ex.sample(frac=1.0,replace=True)
    #print(resample)
    y_t=resample.true
    y_p=resample.pred
    #print(y_t)
    acc_list.append(metrics.accuracy_score(y_t, y_p))
    prec_list.append(metrics.precision_score(y_t, y_p, average='weighted'))
    rec_list.append(metrics.recall_score(y_t, y_p, average='weighted'))
    f1_list.append(metrics.f1_score(y_t, y_p, average='weighted'))
    #print("prec1:", i, prec_list[i])
print(np.median(acc_list),np.percentile(acc_list,2.5), np.percentile(acc_list,97.5))    
print(np.median(prec_list),np.percentile(prec_list,2.5), np.percentile(prec_list,97.5))
print(np.median(rec_list),np.percentile(rec_list,2.5), np.percentile(rec_list,97.5))
print(np.median(f1_list),np.percentile(f1_list,2.5), np.percentile(f1_list,97.5))

0.955 0.941 0.966
0.957764685729249 0.945349061539703 0.9682269841706762
0.955 0.941 0.966
0.9550453282516265 0.9414218133875186 0.9662327272522252


In [231]:
# evaluation of performance on test dataset

In [791]:
#print(y_test)
y_labels1 = to_categorical(np.asarray(y_predict1),num_classes=None) #real to vector
#ym=outm
#y_m=np.argmax(y_labels1,axis=1)
#print("y_m",y_m)

y_pred2=np.argmax(y_labels1,axis=1) #vector to scalar
y_p1=np.argmax(y_test,axis=1) #vecor to scalar
print(y_p1)

print("acc1:",metrics.accuracy_score(y_p1, y_pred2))
print("prec1:",metrics.precision_score(y_p1, y_pred2, average='weighted'))
print("recall1:",metrics.recall_score(y_test, y_labels1, average='weighted'))
print("fscore1:",metrics.f1_score(y_test, y_labels1, average='weighted'))

#print(y_predict1)

[2 7 7 1 0 1 3 0 9 1 2 8 6 3 5 0 7 1 8 9 7 2 0 6 6 2 0 1 6 1 2 7 8 1 0 1 4
 6 6 9 7 2 2 9 4 2 6 2 2 9 4 9 3 2 8 0 6 5 9 4 0 0 8 6 3 6 2 8 2 4 0 3 4 1
 7 9 7 4 9 9 6 0 5 0 4 6 7 0 6 2 7 4 9 0 3 7 6 2 7 8 8 0 9 9 9 6 2 4 2 9 9
 4 7 8 0 2 0 6 4 9 4 2 1 1 3 6 4 7 7 6 9 3 2 7 8 1 7 3 2 7 1 5 2 7 2 5 5 3
 9 5 6 9 7 3 4 1 2 4 9 7]
acc1: 0.6
prec1: 0.6468777494972349
recall1: 0.6
fscore1: 0.6076053063448641


In [131]:
y_true=pd.DataFrame(y_p1,columns=["true"])
y_pred1=pd.DataFrame(y_predict2,columns=["pred"])
y_true1=y_true.reset_index(drop=True)
#print(y_pred1)
#print(y_true1)

In [132]:
y_true_pred=pd.concat([y_true1,y_pred1],axis=1)
y_true_pred.columns=["true","pred"]
#print(y_true_pred)

In [133]:
acc_list=list()
prec_list=list()
rec_list=list()
f1_list=list()
for i in range(1000):
    resample=y_true_pred.sample(frac=1.0,replace=True)
    #print(resample)
    y_t=resample.true
    y_p=resample.pred
    #print(y_t)
    acc_list.append(metrics.accuracy_score(y_t, y_p))
    prec_list.append(metrics.precision_score(y_t, y_p, average='weighted'))
    rec_list.append(metrics.recall_score(y_t, y_p, average='weighted'))
    f1_list.append(metrics.f1_score(y_t, y_p, average='weighted'))
    #print("prec1:", i, prec_list[i])
print(np.median(acc_list),np.percentile(acc_list,2.5), np.percentile(acc_list,97.5))    
print(np.median(prec_list),np.percentile(prec_list,2.5), np.percentile(prec_list,97.5))
print(np.median(rec_list),np.percentile(rec_list,2.5), np.percentile(rec_list,97.5))
print(np.median(f1_list),np.percentile(f1_list,2.5), np.percentile(f1_list,97.5))

0.958436944937833 0.9509680284191829 0.9655417406749556
0.9587141075816155 0.9511845912199828 0.9655747032065876
0.958436944937833 0.9509680284191829 0.9655417406749556
0.9583682232616992 0.9508007740328707 0.9654655468221037


In [134]:
from sklearn.metrics import classification_report

In [135]:
#list1=set(list(y_test))
#res=classification_report(y_test, y_labels1, labels=list1, digits=4, output_dict=True)
res=classification_report(y_test, y_labels1, digits=4)
print(res)

              precision    recall  f1-score   support

           0     0.9957    0.9665    0.9809       239
           1     0.9780    0.9944    0.9861       179
           2     0.9902    1.0000    0.9951       203
           3     0.9878    1.0000    0.9939       162
           4     0.9150    0.9826    0.9476       230
           5     0.9750    0.9653    0.9701       202
           6     1.0000    1.0000    1.0000       147
           7     0.9563    0.9808    0.9684       156
           8     0.9357    0.9125    0.9240       606
           9     0.9461    0.9407    0.9434       691

   micro avg     0.9584    0.9584    0.9584      2815
   macro avg     0.9680    0.9743    0.9709      2815
weighted avg     0.9586    0.9584    0.9584      2815
 samples avg     0.9584    0.9584    0.9584      2815



In [ ]:
# word2vec cnn model

In [903]:
from keras.layers import Embedding

In [904]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
print(len(model_w2v.wv.index2word))

352196


In [905]:
for word, i in word_index.items():
    #print(word,i)
    if word in model_w2v.wv.index2word:
        embedding_matrix[i] = np.asarray(model_w2v[word],dtype='float32')

embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [906]:
embedding_matrix.shape

(2254, 256)

In [907]:
labels.shape

(11260, 10)

In [908]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 64, 256)           577024    
_________________________________________________________________
dropout_35 (Dropout)         (None, 64, 256)           0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 32, 512)           0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_69 (Dense)             (None, 256)               4194560   
_________________________________________________________________
dense_70 (Dense)             (None, 10)                2570      
Total para

In [909]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(x_train, y_train, batch_size=64, epochs=6, verbose=1, validation_data=(x_test, y_test)) 
score = model.evaluate(x_test,y_test, verbose=0)
print(score)

Train on 9008 samples, validate on 2252 samples
Epoch 1/6
9008/9008 [==============================] - 21s 2ms/step - loss: 1.1106 - acc: 0.6348 - val_loss: 0.5352 - val_acc: 0.8419
Epoch 2/6
9008/9008 [==============================] - 21s 2ms/step - loss: 0.3404 - acc: 0.8918 - val_loss: 0.3254 - val_acc: 0.8956
Epoch 3/6
9008/9008 [==============================] - 21s 2ms/step - loss: 0.1775 - acc: 0.9423 - val_loss: 0.2821 - val_acc: 0.9094
Epoch 4/6
9008/9008 [==============================] - 21s 2ms/step - loss: 0.1065 - acc: 0.9651 - val_loss: 0.2558 - val_acc: 0.9232
Epoch 5/6
9008/9008 [==============================] - 21s 2ms/step - loss: 0.0736 - acc: 0.9789 - val_loss: 0.2658 - val_acc: 0.9192
Epoch 6/6
9008/9008 [==============================] - 21s 2ms/step - loss: 0.0553 - acc: 0.9838 - val_loss: 0.2757 - val_acc: 0.9258
[0.2756943941433942, 0.925843694599653]


In [47]:
# word2vec sklearn

In [526]:
import pandas as pd

In [527]:
def get_mean_vector(vectors): #transfer vectors to 
    veclist=[]
    for line in vectors:
        vec_df=pd.DataFrame(line)
        #print(len(line))
        vec_mean=vec_df.mean(axis=0).values
        veclist.append(vec_mean)
    return veclist

In [528]:
w2v_list=get_mean_vector(vectors_w2v) #word2vec sklearn
w2v_ex_list=get_mean_vector(vectors_w2v_ex)

In [530]:
print(len(w2v_list), len(w2v_list[0]))
print(len(w2v_ex_list), len(w2v_ex_list[0]))

14075 256
1000 256


In [611]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
x=pd.DataFrame(w2v_list)
y2=pd.DataFrame(data.tcms_code)

x_train, x_test, y_train, y_test=train_test_split(x,y2,test_size=0.25,random_state=0)
dts=train_test_split(x,y2,test_size=0.25,random_state=0)

x_ex=pd.DataFrame(w2v_ex_list)
y_ex=pd.DataFrame(data_ex.tcms_code)


#x.to_excel("./data/dataset/x_w2v.xls", header=True, index=False,  encoding="utf-8")
#y2.to_excel("./data/dataset/y_w2v.xls", header=True, index=False,  encoding="utf-8")
#x_ex.to_excel("./data/dataset/x_ex_w2v.xls", header=True, index=False,  encoding="utf-8")
#y_ex.to_excel("./data/dataset/y_ex_w2v.xls", header=True, index=False,  encoding="utf-8")

In [612]:
x1=pd.read_excel("./data/dataset/x_w2v.xls")
y1=pd.read_excel("./data/dataset/y_w2v.xls")
x1_ex=pd.read_excel("./data/dataset/x_ex_w2v.xls")
y1_ex=pd.read_excel("./data/dataset/y_ex_w2v.xls")
print(x1.shape)
print(y1.shape)
print(x1_ex.shape)
print(y1_ex.shape)

(14075, 256)
(14075, 1)
(1000, 256)
(1000, 1)


In [538]:
ym=pd.DataFrame(outm)
ym_ex=pd.DataFrame(outm_ex)

ym.to_excel("./data/dataset/ym_w2v.xls", header=True, index=False,  encoding="utf-8")
ym_ex.to_excel("./data/dataset/ym_ex_w2v.xls", header=True, index=False,  encoding="utf-8")

In [540]:
ym1=pd.read_excel("./data/dataset/ym_w2v.xls")
ym1_ex=pd.read_excel("./data/dataset/ym_ex_w2v.xls")
print(ym1.shape)
print(ym1_ex.shape)

(14075, 9)
(1000, 9)


In [595]:
from sklearn.ensemble import RandomForestClassifier
def test_rf(dts):
    cls = RandomForestClassifier(n_estimators=1000)
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    y_pred_ex=cls.predict(x_ex)
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(dts[3], y_predict))
    print("prec1:",metrics.precision_score(dts[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dts[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dts[3], y_predict, average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3], average='micro'))
    #print(y_predict) 
    
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    
    print(y_predict)
    return y_predict

In [596]:
 y_predict1=test_rf(dts)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


acc1: 0.9394714407502132
prec1: 0.9406155448433985
recall1: 0.9394714407502132
fscore1: 0.9392885492206207
acc_ex: 0.962
prec_ex: 0.9623615884920741
rec_ex: 0.962
f1_ex: 0.9617716305961178
[9 1 8 ... 6 7 0]


In [598]:
y_predict1.shape

(3519,)

In [600]:
dts[3].shape

(3519, 1)

In [607]:
#y_true=pd.DataFrame(dts[3],columns=["true"])
#print(dts[3])
y_pred1=pd.DataFrame(y_predict1,columns=["pred"])
#print(y_pred1)
#y_ture=list(y_test)
#print(y_test)
y_true1=dts[3].reset_index(drop=True)
#print(y_true1)

y_true_pred=pd.concat([y_true1,y_pred1],axis=1)
y_true_pred.columns=["true","pred"]
#print(y_true_pred)

      true  pred
0        9     9
1        1     1
2        8     8
3        4     4
4        9     9
5        1     1
6        0     0
7        1     1
8        2     2
9        4     4
10       9     9
11       8     8
12       8     8
13       8     2
14       9     9
15       9     9
16       9     9
17       8     8
18       3     3
19       3     3
20       0     0
21       9     9
22       8     8
23       4     4
24       4     4
25       8     8
26       4     4
27       3     3
28       8     8
29       2     2
...    ...   ...
3489     9     9
3490     8     8
3491     9     9
3492     0     0
3493     2     2
3494     9     9
3495     9     9
3496     9     9
3497     2     2
3498     8     8
3499     9     9
3500     3     3
3501     9     9
3502     9     9
3503     5     5
3504     5     5
3505     2     2
3506     8     8
3507     9     9
3508     8     8
3509     9     9
3510     6     6
3511     9     9
3512     2     2
3513     9     9
3514     8     8
3515     7    

In [608]:
acc_list=list()
prec_list=list()
rec_list=list()
f1_list=list()
for i in range(1000):
    resample=y_true_pred.sample(frac=1.0,replace=True)
    #print(resample)
    y_t=resample.true
    y_p=resample.pred
    #print(y_t)
    acc_list.append(metrics.accuracy_score(y_t, y_p))
    prec_list.append(metrics.precision_score(y_t, y_p, average='weighted'))
    rec_list.append(metrics.recall_score(y_t, y_p, average='weighted'))
    f1_list.append(metrics.f1_score(y_t, y_p, average='weighted'))
    #print("prec1:", i, prec_list[i])
print(np.median(acc_list),np.percentile(acc_list,2.5), np.percentile(acc_list,97.5))    
print(np.median(prec_list),np.percentile(prec_list,2.5), np.percentile(prec_list,97.5))
print(np.median(rec_list),np.percentile(rec_list,2.5), np.percentile(rec_list,97.5))
print(np.median(f1_list),np.percentile(f1_list,2.5), np.percentile(f1_list,97.5))

0.9397556123898835 0.9312304631997727 0.9468599033816425
0.9410840354054404 0.9331467572381069 0.9480129972233186
0.9397556123898835 0.9312304631997727 0.9468599033816425
0.9396254375003966 0.931124207197274 0.9467799223970718


In [613]:
from sklearn.neural_network import MLPClassifier
def test_ann(dts):
    cls=MLPClassifier(alpha=1e-5,hidden_layer_sizes=(256,512,256,128,64,32),random_state=0)
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    y_pred_ex=cls.predict(x_ex)
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(dts[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dts[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dts[3], y_predict, average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3])) 
    
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))
    #print(y_predict) 

In [614]:
 test_ann(dts)

d:\code\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc1: 0.9323671497584541
prec1: 0.9326740811050738
recall1: 0.9323671497584541
fscore1: 0.9319163035746149
acc_ex: 0.95
prec_ex: 0.951100088181693
rec_ex: 0.95
f1_ex: 0.9493563224346927


In [615]:
from xgboost import XGBClassifier

def test_xgboost(dts):
    cls = XGBClassifier()
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    y_pred_ex=cls.predict(x_ex)
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(dts[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dts[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dts[3], y_predict, average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3])) 
    
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))    
    #print(y_predict) 

In [616]:
test_xgboost(dts)

d:\code\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\code\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc1: 0.8832054560954816
prec1: 0.8844589716994606
recall1: 0.8832054560954816
fscore1: 0.8832090603085965
acc_ex: 0.912
prec_ex: 0.9153375295760466
rec_ex: 0.912
f1_ex: 0.9128070365620301


In [617]:
from sklearn.neighbors import KNeighborsClassifier
def test_knn(dts):
    cls = KNeighborsClassifier()
    #cls.fit(dts[0],dts[2])
    #y_predict=cls.predict(dts[2])
    #print("acc:",metrics.accuracy_score(y_predict,dts[3]))
    
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    y_pred_ex=cls.predict(x_ex)
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(dts[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dts[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dts[3], y_predict, average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3])) 
    
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))    
    #print(y_predict) 

In [618]:
test_knn(dts)

d:\code\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


acc1: 0.8485365160556977
prec1: 0.8489887855906272
recall1: 0.8485365160556977
fscore1: 0.8478838876381439
acc_ex: 0.898
prec_ex: 0.9014113640202703
rec_ex: 0.898
f1_ex: 0.8980135081336035


In [619]:
from sklearn import svm
def test_svc(dts):
    cls=svm.SVC(kernel="linear")
    cls.fit(dts[0],dts[2])
    y_predict=cls.predict(dts[1])
    #y_predict0=cls.predict(dts[0])
    y_pred_ex=cls.predict(x_ex)
    #print("acc0:",metrics.accuracy_score(y_predict0,dts[2]))
    print("acc1:",metrics.accuracy_score(y_predict,dts[3]))
    print("prec1:",metrics.precision_score(dts[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dts[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dts[3], y_predict, average='weighted'))
    #print("auc1:",metrics.roc_auc_score(y_predict,dts[3])) 
    #print(y_predict) 
    
    print("acc_ex:",metrics.accuracy_score(y_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(y_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(y_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(y_ex, y_pred_ex, average='weighted'))      

In [620]:
test_svc(dts)

d:\code\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc1: 0.817277635691958
prec1: 0.8245302301125366
recall1: 0.817277635691958
fscore1: 0.8161642675144759
acc_ex: 0.779
prec_ex: 0.8047899086548251
rec_ex: 0.779
f1_ex: 0.7826982283472426


In [886]:
# word2vec sklearn for tcm element

In [628]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
xm=pd.DataFrame(w2v_list)
ym=pd.DataFrame(outm)
x_train, x_test, y_train, y_test=train_test_split(xm,ym,test_size=0.25,random_state=0)
dtsm=train_test_split(xm,ym,test_size=0.25,random_state=0)

#x_ex=x_ex
ym_ex=pd.DataFrame(outm_ex)

In [629]:
xm.shape

(14075, 256)

In [630]:
ym.shape

(14075, 9)

In [636]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [637]:
from sklearn.ensemble import RandomForestClassifier
def test_rfm(dtsm):
    rfm=RandomForestClassifier(n_estimators=100)
    clsm=MultiOutputClassifier(rfm)
    clsm.fit(dtsm[0],dtsm[2])
    y_predict=clsm.predict(dtsm[1])
    y_pred_ex=clsm.predict(x_ex)
    
    print("acc1:",metrics.accuracy_score( y_predict,dtsm[3]))  
    print("prec1:",metrics.precision_score(dtsm[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dtsm[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dtsm[3], y_predict, average='weighted'))
    
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))  
    
    res=classification_report(dtsm[3],  y_predict, digits=4)
    print(res)
    #print(y_predict) 

In [638]:
test_rfm(dtsm)

acc1: 0.9084967320261438
prec1: 0.9919103110275276
recall1: 0.9608994240141782
fscore1: 0.9755981087028768
acc_ex: 0.973
prec_ex: 0.9953956174341454
rec_ex: 0.9877017678708686
f1_ex: 0.9913826110688292
              precision    recall  f1-score   support

           0     0.9920    0.9404    0.9655      1578
           1     0.9940    0.9345    0.9633       534
           2     0.9924    0.9924    0.9924       662
           3     0.9674    0.8285    0.8926      1038
           4     0.9943    0.9860    0.9901       713
           5     0.9895    0.9653    0.9773       490
           6     1.0000    1.0000    1.0000      3519
           7     0.9739    0.9491    0.9613       275
           8     0.9910    1.0000    0.9955       219

   micro avg     0.9923    0.9609    0.9764      9028
   macro avg     0.9883    0.9551    0.9709      9028
weighted avg     0.9919    0.9609    0.9756      9028
 samples avg     0.9927    0.9632    0.9725      9028



In [639]:
from sklearn.neural_network import MLPClassifier
def test_mlpm(dtsm):      
    mlp=MLPClassifier(alpha=1e-5,hidden_layer_sizes=(256,512,256,128,64,32),random_state=0)
    clsm=MultiOutputClassifier(mlp)
    #clsm.fit(dtsm[0],dtsm[2])
    #y_predict=clsm.fit(dtsm[0],dtsm[1]).predict(dtsm[2])
    #y_predict=clsm.predict(dtsm[1])
    #print("acc:",metrics.accuracy_score(y_predict,dtsm[3]))
    clsm.fit(dtsm[0],dtsm[2])
    y_predict=clsm.predict(dtsm[1])
    y_pred_ex=clsm.predict(x_ex)
    #y_predict=clsm.fit(dtsm[0],dtsm[2]).predict(dtsm[1])
    #y_predict0=clsm.predict(dtsm[0])
    #y_predict=clsm.predict(dtsm[1])
    #print("acc0:",metrics.accuracy_score(dts[2], y_predict0))  
    print("acc1:",metrics.accuracy_score( y_predict,dtsm[3]))  
    print("prec1:",metrics.precision_score(dtsm[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dtsm[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dtsm[3], y_predict, average='weighted'))
    
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))   

In [640]:
test_mlpm(dtsm)

acc1: 0.9050866723500994
prec1: 0.975047946274982
recall1: 0.9758529020824103
fscore1: 0.9752786475868975
acc_ex: 0.952
prec_ex: 0.9891125335127247
rec_ex: 0.9880860876249039
f1_ex: 0.9884158433989122


In [641]:
from xgboost import XGBClassifier
def test_xgboost(dtsm):
    xgbm = XGBClassifier()
    clsm=MultiOutputClassifier(xgbm)
    #clsm.fit(dtsm[0],dtsm[2])
    #y_predict=clsm.predict(dtsm[1])
    #print("acc:",metrics.accuracy_score(y_predict,dtsm[3]))
    #print(y_predict) 
    clsm.fit(dtsm[0],dtsm[2])
    y_predict=clsm.predict(dtsm[1])
    y_pred_ex=clsm.predict(x_ex)
    #y_predict=clsm.fit(dtsm[0],dtsm[2]).predict(dtsm[1])
    #y_predict0=clsm.predict(dtsm[0])
    #y_predict=clsm.predict(dtsm[1])
    #print("acc0:",metrics.accuracy_score(dts[2], y_predict0))  
    print("acc1:",metrics.accuracy_score( y_predict,dtsm[3]))  
    print("prec1:",metrics.precision_score(dtsm[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dtsm[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dtsm[3], y_predict, average='weighted'))
    
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))   

In [642]:
 test_xgboost(dtsm)

acc1: 0.7703893151463483
prec1: 0.9666584429151647
recall1: 0.9044085068675233
fscore1: 0.9333696981775362
acc_ex: 0.798
prec_ex: 0.9702001999102109
rec_ex: 0.9227517294388932
f1_ex: 0.9444549973673023


In [643]:
from sklearn.neighbors import KNeighborsClassifier
def test_knnm(dtsm):
    knnm=KNeighborsClassifier()
    clsm=MultiOutputClassifier(knnm)
    #clsm.fit(dtsm[0],dtsm[1])
    #y_predict=clsm.predict(dtsm[2])
    #print("acc:",metrics.accuracy_score(y_predict,dtsm[3]))
    #print(y_predict)
    
    clsm.fit(dtsm[0],dtsm[2])
    y_predict=clsm.predict(dtsm[1])
    y_pred_ex=clsm.predict(x_ex)
    #y_predict=clsm.fit(dtsm[0],dtsm[2]).predict(dtsm[1])
    #y_predict0=clsm.predict(dtsm[0])
    #y_predict=clsm.predict(dtsm[1])
    #print("acc0:",metrics.accuracy_score(dts[2], y_predict0))  
    print("acc1:",metrics.accuracy_score( y_predict,dtsm[3]))  
    print("prec1:",metrics.precision_score(dtsm[3], y_predict, average='weighted'))
    print("recall1:",metrics.recall_score(dtsm[3], y_predict, average='weighted'))
    print("fscore1:",metrics.f1_score(dtsm[3], y_predict, average='weighted'))
    
    print("acc_ex:",metrics.accuracy_score(ym_ex, y_pred_ex))
    print("prec_ex:",metrics.precision_score(ym_ex, y_pred_ex, average='weighted'))
    print("rec_ex:",metrics.recall_score(ym_ex, y_pred_ex, average='weighted'))
    print("f1_ex:",metrics.f1_score(ym_ex, y_pred_ex, average='weighted'))   

In [644]:
test_knnm(dtsm)

acc1: 0.8414322250639387
prec1: 0.9380038791496662
recall1: 0.9254541426672575
fscore1: 0.931291900447917
acc_ex: 0.892
prec_ex: 0.9640784654380703
rec_ex: 0.9473481936971561
f1_ex: 0.9546746215667181


In [816]:
# word2vec cnn for multi labels

In [662]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [663]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [664]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()

In [317]:
#ym=np.array(outm)
#ym_out=mlb.fit_transform(ym)
#print(ym_out)

In [683]:
ym=pd.DataFrame(outm)

all_texts=cps1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#labels = to_categorical(np.asarray(y),num_classes=None)
print('Shape of data tensor:', datatxt.shape)
print('Shape of label tensor:', ym.shape)
print(word_index)

Found 2362 unique tokens.
Shape of data tensor: (14075, 64)
Shape of label tensor: (14075, 9)
{' ': 1, '咳嗽': 2, '咳痰': 3, '患者': 4, '面色': 5, '精神': 6, '气喘': 7, '清晰': 8, '加重': 9, '为主': 10, '反复': 11, '言语': 12, '正常': 13, '胸闷': 14, '苔': 15, '舌质': 16, '痰': 17, '形体': 18, '营养': 19, '发育': 20, '后': 21, '自如': 22, '应答': 23, '天': 24, '结合': 25, '表情': 26, '少华': 27, '气促': 28, '适中': 29, '少神': 30, '诉': 31, '活动': 32, '肺癌': 33, '脉滑': 34, '伴': 35, '舌脉': 36, '行动自如': 37, '阵发性': 38, '神清': 39, '咯痰': 40, '咳声': 41, '声音洪亮': 42, '慢性阻塞性肺病伴感染': 43, '红': 44, '发热': 45, '中等': 46, '二便': 47, '稍': 48, '精神不振': 49, '时有': 50, '肺部感染': 51, '痛苦': 52, '白腻': 53, '再发': 54, '': 55, '轻微': 56, '不易': 57, '黄腻': 58, '舌红': 59, '良好': 60, '余年': 61, '有神': 62, '淡红': 63, '脉浮': 64, '咳出': 65, '年': 66, '调': 67, '脉滑数': 68, '乏力': 69, '舌淡': 70, '清': 71, '支气管哮喘': 72, '语声低微': 73, '呼吸': 74, '粘': 75, '双目': 76, '苔薄': 77, '明显': 78, '欠佳': 79, '动则': 80, '月余': 81, '淡': 82, '粘痰': 83, '差': 84, '语声': 85, '黄': 86, '气息': 87, '咯出': 88, '脉细': 89, '岁': 90, '呈': 91, '

In [684]:
from sklearn.model_selection import train_test_split
x=datatxt
#ym=pd.DataFrame(outm)
x_train, x_test, y_train, y_test=train_test_split(x,ym,test_size=0.20,random_state=0)

In [685]:
y_test.head()

,tan,feng,han,ren,qixu,yinxu,fei,pi,shen
5954,0,0,1,0,0,0,1,0,0
4150,0,1,0,1,0,0,1,0,0
5325,0,0,1,0,0,0,1,0,0
3188,0,1,1,0,0,0,1,0,0
3870,0,1,1,0,0,0,1,0,0


In [686]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
#EMBEDDING_DIM = 128

In [687]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
#model.add(Dense(ym.shape[1], activation='softmax')) #multiclass
model.add(Dense(ym.shape[1], activation='sigmoid')) #multilabel
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 64, 256)           604928    
_________________________________________________________________
dropout_16 (Dropout)         (None, 64, 256)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 32, 512)           0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               4194560   
_________________________________________________________________
dense_32 (Dense)             (None, 9)                 2313      
Total para

In [688]:
#from keras import metrics
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
#[metrics.mae, metrics.categorical_accuracy]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"]) 
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_split=0.2, shuffle=True)
score = model.evaluate(x_test,y_test, verbose=0)
print(score[1])

y_pred1=model.predict(x_test)

Train on 9008 samples, validate on 2252 samples
Epoch 1/4
9008/9008 [==============================] - 27s 3ms/step - loss: 0.1484 - acc: 0.9381 - val_loss: 0.0397 - val_acc: 0.9866
Epoch 2/4
9008/9008 [==============================] - 28s 3ms/step - loss: 0.0292 - acc: 0.9906 - val_loss: 0.0229 - val_acc: 0.9929
Epoch 3/4
9008/9008 [==============================] - 27s 3ms/step - loss: 0.0136 - acc: 0.9957 - val_loss: 0.0185 - val_acc: 0.9941
Epoch 4/4
9008/9008 [==============================] - 28s 3ms/step - loss: 0.0074 - acc: 0.9976 - val_loss: 0.0193 - val_acc: 0.9951
0.9943556452940878


In [689]:
import sklearn.metrics as metrics

In [427]:
y_pred1=model.predict(x_test)
#print("acc1:",metrics.accuracy_score(y_test, y_pred1))

#print(y_test)
#print(y_pred1)

#y_labels1 = to_categorical(np.asarray(y_predict1),num_classes=None)

#y_m1=np.argmax(y_pred1,axis=1)
#y_l1 = to_categorical(np.asarray(y_pred1),num_classes=None)
#print(y_l1)

In [690]:
def get_multilabel_output(arr,threshold):
    threshold1=threshold
    arr1=list()
    LEN=len(arr)
    
    for i in range(LEN):
        if arr[i]>=threshold1:
            arr1.append(1)
        else: arr1.append(0)
    return arr1

In [691]:
def get_multilabel(y_pred):
    outlist=list()
    for val in y_pred:
        valist= get_multilabel_output(val,0.7)
        outlist.append(valist)
    return outlist

In [692]:
arr1=get_multilabel(y_pred1)
arrdf=pd.DataFrame(arr1)

In [693]:
print("acc1:",metrics.accuracy_score(y_test, arrdf))
print("prec1:",metrics.precision_score(y_test, arrdf, average='weighted'))
print("rec1:",metrics.recall_score(y_test, arrdf, average='weighted'))
print("f1:",metrics.f1_score(y_test, arrdf, average='weighted'))

res=classification_report(y_test, arrdf, digits=4)
print(res)

acc1: 0.9559502664298402
prec1: 0.9922375399528273
rec1: 0.986335403726708
f1: 0.9891990667394598
              precision    recall  f1-score   support

           0     0.9935    0.9951    0.9943      1233
           1     0.9953    0.9770    0.9861       435
           2     0.9960    1.0000    0.9980       503
           3     0.9675    0.9174    0.9418       811
           4     0.9871    0.9935    0.9903       616
           5     0.9975    0.9801    0.9887       403
           6     1.0000    1.0000    1.0000      2815
           7     0.9771    0.9922    0.9846       258
           8     0.9826    0.9883    0.9854       171

   micro avg     0.9924    0.9863    0.9893      7245
   macro avg     0.9885    0.9826    0.9855      7245
weighted avg     0.9922    0.9863    0.9892      7245
 samples avg     0.9935    0.9877    0.9891      7245

